In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import os
import zipfile

In [ ]:
# Define paths for the zip file and its extraction location
# Update this path based on your file location in Colab
zip_path = '/content/faces_224.zip'  # Path to the uploaded zip file
unzip_path = '/content'   # Path where files will be unzipped

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_path)

# Update the image path
image_path = unzip_path

In [ ]:
def create_resnet_model(input_shape=(224, 224, 3)):
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    base_model.trainable = False  # Freeze the base model initially

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid')
    ])

    return model


def create_data_augmentation():
    return ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        brightness_range=[0.8, 1.2],   # Added brightness adjustment
        zoom_range=0.2,                # Added zoom
        preprocessing_function=tf.keras.applications.resnet50.preprocess_input
    )


def train_resnet_model(model, train_generator, val_generator, steps_per_epoch, validation_steps, epochs=20):
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
    ]

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch,
                        validation_data=val_generator, validation_steps=validation_steps,
                        callbacks=callbacks)

    return history


def fine_tune_resnet_model(model, train_generator, val_generator, steps_per_epoch, validation_steps, epochs=10):
    base_model = model.layers[0]  # Get the base model (ResNet50)
    base_model.trainable = True   # Unfreeze the base model for fine-tuning

    # Freeze all layers except the last few (fine-tune only these layers)
    for layer in base_model.layers[:-10]:  # Adjust the number of layers as needed
        layer.trainable = False

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(train_generator, epochs=epochs, steps_per_epoch=steps_per_epoch,
                        validation_data=val_generator, validation_steps=validation_steps,
                        callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)])

    return history


def retrieve_generator(set_name, image_path, target_size=(224, 224), batch_size=32):
    while True:
        for start in range(0, len(set_name), batch_size):
            images, labels = [], []
            end = min(start + batch_size, len(set_name))
            batch_set = set_name[start:end]

            for img, imclass in zip(batch_set['videoname'], batch_set['label']):
                img_path = os.path.join(image_path, img[:-4] + '.jpg')
                image = cv2.imread(img_path)

                if image is not None:
                    image = cv2.resize(image, target_size)
                    images.append(image)
                    labels.append(1 if imclass == 'FAKE' else 0)
                else:
                    print(f"Warning: Image not found or cannot be read - {img_path}")

            # Yield batch
            yield np.array(images), np.array(labels)

In [ ]:
# Main execution
if __name__ == "__main__":
    # Define dataset paths (using your existing paths)
    metadata_path = r"/content/metadata.csv"
    image_path = r"/content/faces_224"

    # Load metadata
    print("Loading metadata...")
    df = pd.read_csv(metadata_path)
    print("Metadata loaded.")

    # Sample and split data
    print("Splitting dataset...")
    real_df = df[df["label"] == "REAL"]
    fake_df = df[df["label"] == "FAKE"]
    sample_size = 10000
    real_df = real_df.sample(sample_size, random_state=42)
    fake_df = fake_df.sample(sample_size, random_state=42)
    sample_meta = pd.concat([real_df, fake_df])

    Train_set, Test_set = train_test_split(sample_meta, test_size=0.2, random_state=42, stratify=sample_meta['label'])
    Train_set, Val_set = train_test_split(Train_set, test_size=0.3, random_state=42, stratify=Train_set['label'])
    print("Dataset split complete.")

    # Initialize generators
    batch_size = 32
    train_generator = retrieve_generator(Train_set, image_path=image_path, batch_size=batch_size )
    val_generator = retrieve_generator(Val_set, image_path=image_path, batch_size=batch_size)
    test_generator = retrieve_generator(Test_set, image_path=image_path, batch_size=batch_size)

    # Calculate steps per epoch
    steps_per_epoch = len(Train_set) // batch_size
    validation_steps = len(Val_set) // batch_size

    # Create model
    print("Creating model...")
    model = create_resnet_model()

    # Train model
    print("Training model...")
    history = train_resnet_model(model, train_generator, val_generator, steps_per_epoch, validation_steps, epochs=10)

    # Fine-tune model
    print("Fine-tuning model...")
    history_ft = fine_tune_resnet_model(model, train_generator, val_generator, steps_per_epoch, validation_steps)

    # Evaluate model
    print("Evaluating model...")
    test_loss, test_accuracy = model.evaluate(test_generator, steps=len(Test_set) // batch_size)
    print(f"Final Test Accuracy: {test_accuracy:.4f}")

    # Save the model
    from google.colab import files

    # Save the model to the current directory
    model.save('deepfake_detector_resnet50.h5')

    # Provide a link to download the model
    files.download('deepfake_detector_resnet50.h5')

    # Plot training history
    plt.figure(figsize=(12, 4))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.plot(history_ft.history['accuracy'], label='Fine-tuning Training Accuracy', linestyle='--')
    plt.plot(history_ft.history['val_accuracy'], label='Fine-tuning Validation Accuracy', linestyle='--')
    plt.title('Training and Fine-tuning Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.plot(history_ft.history['loss'], label='Fine-tuning Training Loss', linestyle='--')
    plt.plot(history_ft.history['val_loss'], label='Fine-tuning Validation Loss', linestyle='--')
    plt.title('Training and Fine-tuning Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()